In [1]:
import os
os.chdir('../')
!pwd

/e/SKILLS/ARTIFICIAL INTELLIGENCE/Krish Naik Academy/Chicken-Disease-Classification


entity

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

configaration

In [11]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
    ):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config

component

In [12]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config


    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)


    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

pipeline

In [13]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    print(prepare_callbacks_config)

    prepare_callback = PrepareCallback(config=prepare_callbacks_config)
    callbacks = prepare_callback.get_tb_ckpt_callbacks()
    print(callbacks)
except Exception as e:
    raise e

[2025-09-15 20:31:08,180: INFO: common]: yaml file: config\config.yaml loaded successfully
[2025-09-15 20:31:08,185: INFO: common]: yaml file: params.yaml loaded successfully
[2025-09-15 20:31:08,187: INFO: common]: created directory at: artifacts
[2025-09-15 20:31:08,191: INFO: common]: created directory at: artifacts\prepare_callbacks\checkpoint_dir
[2025-09-15 20:31:08,193: INFO: common]: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir
PrepareCallbacksConfig(root_dir=WindowsPath('artifacts/prepare_callbacks'), tensorboard_root_log_dir=WindowsPath('artifacts/prepare_callbacks/tensorboard_log_dir'), checkpoint_model_filepath=WindowsPath('artifacts/prepare_callbacks/checkpoint_dir/model.h5'))
[<tensorflow.python.keras.callbacks.TensorBoard object at 0x0000018D6F9CDF10>, <tensorflow.python.keras.callbacks.ModelCheckpoint object at 0x0000018D6F9CDEB0>]


In [14]:
import time
timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
f"tb_logs_at_{timestamp}"

'tb_logs_at_2025-09-15-20-38-16'